<a href="https://colab.research.google.com/github/varnikaaavyas/Time-Series-Analysis/blob/main/SAIMA_forecasting_method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Name - Varnika** \
**Enrollnment No - E22MCAG0015**

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
from statsmodels.graphics.tsaplots import plot_pacf,plot_acf
from statsmodels.tsa.stattools import pacf
from statsmodels.tsa.stattools import acf
from statsmodels.tsa.statespace.sarimax import SARIMAX
# import pmdarima as pm
from sklearn.model_selection import train_test_split
# from pmdarima import auto_arima

In [ ]:
df = pd.read_csv("/content/AirPassengers.csv")
data = df.rename( columns = {'#Passengers':'passengers'})
data

,Month,passengers
0,1949-01,112
1,1949-02,118
2,1949-03,132
3,1949-04,129
4,1949-05,121
...,...,...
139,1960-08,606
140,1960-09,508
141,1960-10,461
142,1960-11,390


In [ ]:
test_series_avg = np.mean(data.passengers)
test_series_avg

280.2986111111111

In [ ]:
test_series_avg
data['sts_avg'] =np.full(shape=len(data.Month), fill_value =test_series_avg , dtype='float')
data

,Month,passengers,sts_avg
0,1949-01,112,280.298611
1,1949-02,118,280.298611
2,1949-03,132,280.298611
3,1949-04,129,280.298611
4,1949-05,121,280.298611
...,...,...,...
139,1960-08,606,280.298611
140,1960-09,508,280.298611
141,1960-10,461,280.298611
142,1960-11,390,280.298611


In [ ]:
fig = px.line(data, x=data.Month, y = data.columns)
fig.show()

In [ ]:
df = pd.read_csv("/content/AirPassengers.csv")
data1 = df.rename( columns = {'#Passengers':'passengers'})
data1['Moving_AVG'] = data['passengers'].rolling(window=3).mean()

data1

,Month,passengers,Moving_AVG
0,1949-01,112,NaN
1,1949-02,118,NaN
2,1949-03,132,120.666667
3,1949-04,129,126.333333
4,1949-05,121,127.333333
...,...,...,...
139,1960-08,606,587.666667
140,1960-09,508,578.666667
141,1960-10,461,525.000000
142,1960-11,390,453.000000


In [ ]:
fig = px.line(data1, data1.Month , data1.columns)
fig.show()

In [ ]:
df = pd.read_csv("/content/AirPassengers.csv")
data2 = df.rename( columns = {'#Passengers':'passengers'})
weights = np.array([0.1,0.2,0.3])
data2['Expomential_WMA'] = data2['passengers'].rolling(window=3).apply(lambda x: np.sum(weights*x))

data2

,Month,passengers,Expomential_WMA
0,1949-01,112,NaN
1,1949-02,118,NaN
2,1949-03,132,74.4
3,1949-04,129,76.9
4,1949-05,121,75.3
...,...,...,...
139,1960-08,606,359.7
140,1960-09,508,335.8
141,1960-10,461,300.5
142,1960-11,390,260.0


In [ ]:
fig = px.line(data2, data2.Month, data2.columns)
fig.show()

In [ ]:
train, test = train_test_split(df, test_size=0.3, shuffle=False)
train.shape

NameError: ignored

In [ ]:
model= pm.auto_arima(train,start_p=0,start_q=0,test ='adf',max_p=5,max_q=5,m=12,d=0, max_d = 3, start_P = 0, max_P = 4, start_Q = 0, max_Q = 4, D = 1, max_D = 3, error_action= 'ignore',suppress_warnings=True,stepwise=True, seasonal=True, trace=True)
print(model.summary())

In [ ]:
#best_model = SARIMAX(train.Passengers, order = (1,0,0), seasonal_order = (0,1,1, 12)).fit()

In [ ]:
#best_model.summary()

In [ ]:
pred = model.predict(n_periods=44)
pred_series = pd.Series(pred)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(train, label='Training')
plt.plot(test, label='Testing')
plt.plot(pred_series, label='Predicted')
plt.legend()
plt.show()

In [ ]:
def performance_metrics(forecast, actual):
    mape = np.mean((np.abs(actual - forecast)/actual)*100)
    me = np.mean(actual - forecast)
    mpe = np.mean(((forecast-actual)/actual)*100)
    mae = np.mean(np.abs(actual - forecast))
    rmse = np.sqrt(np.mean(forecast - actual)**2)
    corr = np.corrcoef(actual, forecast)[0,1]
    mins = np.amin(np.hstack([actual[:,None], forecast[:, None]]), axis = 1)
    maxs = np.amax(np.hstack([actual[:,None], forecast[:, None]]), axis = 1)
    minmax = 1 - np.mean(mins/maxs)
    acf1 = acf(actual - forecast)[1]
    return({'mape': mape, 'me': me, 'mae': mae, 'mpe':mpe, 'rmse': rmse, 'acf1': acf1,
    'corr': corr, 'minmax': minmax})

In [ ]:
test1 = test["#Passengers"].values
pred1 = pred_series.values

In [ ]:
performance_metrics(pred1, test1)

In [ ]:
time_series_avg = np.mean(df.)